In [ ]:
### Calculates and plots the 2D AMOC streamfunction in sig- and z- space, for all models listed in the run dictionary (run_dict)
### The AMOCsig calculation is done a number of ways, mostly for demonstration purposes, but also to compare the different ways of calculating:  
### 1) Sigma2 using online output
### 2) Sigma0 using offline output
### 3) Sigma2 using offline output (for comparison of the offline method to the online truth)

In [ ]:
%matplotlib inline

import sys
sys.path.insert(0, '/home/Matthew.Thomas/python_code/python_pkgs/MOC_funcs')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
from numba import njit
import sectionate
from glob import glob
import momlevel
from xhistogram.xarray import histogram
import MOC_funcs
from cmip_basins.basins import generate_basin_codes

### Set up a dask server, if needed:

In [ ]:
# import dask
# from dask.distributed import Client
# from dask_jobqueue import SLURMCluster
# dask.config.set(**{'array.slicing.split_large_chunks': False})

# portdash = 18224

# cluster = SLURMCluster(
#     queue="batch",
#     cores=8,
#     processes=4,
#     project="gfdl_o",
#     memory="48GB",
#     walltime="16:00:00",
#     local_directory="$TMPDIR",
#     death_timeout=120,
#     scheduler_options={"dashboard_address":f":{portdash}"},
# )

# client = Client(cluster)
# client

In [ ]:
# cluster.scale(jobs=10)

# First calculate online streamfunctions

In [ ]:
run_dict = {
                'odiv209' : {
                             'density_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/ts/annual/10yr/",
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.0211*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv1' :   {
                             'density_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_rho2/ts/annual/5yr/",
                             'depth_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ts/annual/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.0211*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv230' : {
                             'density_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/ts/annual/10yr/",
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.0211*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv231' : {
                             'density_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/ts/annual/10yr/",
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.1960*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv232' : {
                             'density_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_rho2/ts/annual/10yr/",
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.2060*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv2' :   {
                             'density_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_rho2/ts/annual/5yr/",
                             'depth_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ts/annual/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.1960*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv2_ssp585' :   {
                             'density_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201803_om4_v1.0.1/CM4_Scenario_ssp585/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_rho2/ts/annual/5yr/",
                             'depth_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201803_om4_v1.0.1/CM4_Scenario_ssp585/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ts/annual/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201803_om4_v1.0.1/CM4_Scenario_ssp585/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual_z/ocean_annual_z.static.nc",
                             'file_str_identifier' : "*.2060*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
            }

In [ ]:
%%time
### AMOCz in CM4hires
ComputeDiag_dict_Online = {}
dmgetout_flag = False
zarr_dir='/xtmp/Matthew.Thomas/work'
if dmgetout_flag: print(f'run the following dmget commands:')
for keys in run_dict:
    ds_sig, grid_sig, moc_sig, vmo_sig_zonmean = MOC_funcs.MOC_basin_latrange_online(dir_vars=run_dict[keys]['density_vars_path'],dir_grid=run_dict[keys]['grid_path'],file_str_identifier=run_dict[keys]['file_str_identifier'],lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],decode_times_flag=False,dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    ds_z, grid_z, moc_z, vmo_z_zonmean = MOC_funcs.MOC_basin_latrange_online(dir_vars=run_dict[keys]['depth_vars_path'],dir_grid=run_dict[keys]['grid_path'],file_str_identifier=run_dict[keys]['file_str_identifier'],lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],z_layer_var=run_dict[keys]['z_layer_var'],z_inter_var=run_dict[keys]['z_inter_var'],decode_times_flag=False,dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    if len(ds_sig)>0:
        AMOCsig_sf=ds_sig.vmo.mean('time').fillna(0).sum('xh').cumsum('rho2_l')/1e6/1030
        AMOCz_sf=ds_z.vmo.mean('time').fillna(0).sum('xh').cumsum('z_l')/1e6/1030    
        ComputeDiag_dict_Online[keys]={
                                'ds_z' : ds_z, 'grid_z' : grid_z , 'moc_z': moc_z, 'vmo_z_zonmean' : vmo_z_zonmean, 'AMOCsig_sf' : AMOCsig_sf,
                                'ds_sig' : ds_sig, 'grid_sig' : grid_sig , 'moc_sig': moc_sig, 'vmo_sig_zonmean' : vmo_sig_zonmean, 'AMOCz_sf' : AMOCz_sf,
                                }

In [ ]:
load_data_flag=1
fig = plt.figure(figsize=(16,len(ComputeDiag_dict_Online.keys())*5))
subplot_num=1
for keys in run_dict:
    
    if load_data_flag:
        ComputeDiag_dict_Online[keys]['AMOCsig_sf']=ComputeDiag_dict_Online[keys]['AMOCsig_sf'].load()
        ComputeDiag_dict_Online[keys]['AMOCz_sf']=ComputeDiag_dict_Online[keys]['AMOCz_sf'].load()
    
    ax = fig.add_subplot(len(ComputeDiag_dict_Online.keys()),2,(subplot_num*2)-1)
    ComputeDiag_dict_Online[keys]['AMOCsig_sf'].plot.contourf(vmin=-10,vmax=24,levels=60,ylim=[1038,1030],cmap='RdBu_r')
    plt.title('AMOCsig for years '+run_dict[keys]['file_str_identifier']+'. Run '+keys)
    plt.xlabel('')
    ax = fig.add_subplot(len(ComputeDiag_dict_Online.keys()),2,subplot_num*2)
    ComputeDiag_dict_Online[keys]['AMOCz_sf'].plot.contourf(vmin=-5,vmax=20,levels=30,ylim=[6000,0],cmap='RdBu_r')
    plt.fill_between(ComputeDiag_dict_Online[keys]['grid_z'].yh,ComputeDiag_dict_Online[keys]['grid_z'].deptho.where(ComputeDiag_dict_Online[keys]['grid_z'].deptho.xh>-80).max(dim='xh'),8000,color='k')
    plt.title('AMOCz for years '+run_dict[keys]['file_str_identifier']+'. Run '+keys)
    plt.xlabel('')
    subplot_num+=1

# Now calculate offline streamfunctions ...

In [ ]:
run_dict = {
                'odiv209' : {
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_month_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_month_z/ocean_month_z.static.nc",
                             'file_str_identifier' : "*.0211*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv1' :   {
                             'depth_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_piControl_C/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'file_str_identifier' : "*.0211*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },

                'odiv231' : {
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'file_str_identifier' : "*.1960*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv232' : {
                             'depth_vars_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'file_str_identifier' : "*.2060*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },    
                'odiv2' :   {
                             'depth_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'file_str_identifier' : "*.1960*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
                'odiv2_ssp585' :   {
                             'depth_vars_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201803_om4_v1.0.1/CM4_Scenario_ssp585/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/",
                             'grid_path' : "/archive/oar.gfdl.cmip6/CM4/warsaw_201803_om4_v1.0.1/CM4_Scenario_ssp585/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc",
                             'file_str_identifier' : "*.2060*",
                             'lat_range' : [40,65],
                             'basin_list': [2,6,7],
                             'z_layer_var' : "z_l",
                             'z_inter_var' : "z_i"
                             },
            }

### ... first in sigma0

In [ ]:
%%time
### AMOCz in CM4hires
ComputeDiag_dict_OfflineSig0 = {}
dmgetout_flag = False
zarr_dir='/xtmp/Matthew.Thomas/work'
if dmgetout_flag: print(f'run the following dmget commands:')
for keys in run_dict:
    ds, grid, moc_sig, rhopoto = MOC_funcs.MOC_basin_latrange_offline(dir_vars=run_dict[keys]['depth_vars_path'],dir_grid=run_dict[keys]['grid_path'],file_str_identifier=run_dict[keys]['file_str_identifier'],nrho=50,rholims=[21,28.1],ref_pres=0,lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],z_layer_var=run_dict[keys]['z_layer_var'],z_inter_var=run_dict[keys]['z_inter_var'],decode_times_flag=True,dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    if len(ds)>0:
        moc_z=ds.vmo.mean('time').fillna(0).sum('xh').cumsum('z_l')/1e6/1030   
        ComputeDiag_dict_OfflineSig0[keys]={
                                'ds' : ds, 'grid': grid, 'moc_sig' : moc_sig, 'rhopoto' : rhopoto, 'moc_z' : moc_z,
                                }

In [ ]:
%%time
load_data_flag=0
fig = plt.figure(figsize=(16,len(ComputeDiag_dict_OfflineSig0.keys())*5))
subplot_num=1
for keys in run_dict:
    
    if load_data_flag:
        ComputeDiag_dict_OfflineSig0[keys]['moc_sig']=ComputeDiag_dict_OfflineSig0[keys]['moc_sig'].load()
        ComputeDiag_dict_OfflineSig0[keys]['moc_z']=ComputeDiag_dict_OfflineSig0[keys]['moc_z'].load()
    
    ax = fig.add_subplot(len(ComputeDiag_dict_OfflineSig0.keys()),2,(subplot_num*2)-1)
    ComputeDiag_dict_OfflineSig0[keys]['moc_sig'].plot.contourf(y='rhopoto_bin',vmin=-10,vmax=24,levels=30,ylim=[28,22],cmap='RdBu_r')
    plt.title('AMOCsig for years '+run_dict[keys]['file_str_identifier']+'. Run '+keys)
    plt.xlabel('')
    ax = fig.add_subplot(len(ComputeDiag_dict_OfflineSig0.keys()),2,subplot_num*2)
    ComputeDiag_dict_OfflineSig0[keys]['moc_z'].plot.contourf(vmin=-5,vmax=20,levels=30,ylim=[6000,0],cmap='RdBu_r')
    plt.fill_between(ComputeDiag_dict_OfflineSig0[keys]['grid'].yh,ComputeDiag_dict_OfflineSig0[keys]['grid']['deptho'].max(dim='xh'),8000,color='k')
    plt.title('AMOCz for years '+run_dict[keys]['file_str_identifier']+'. Run '+keys)
    plt.xlabel('')
    subplot_num+=1

### ... now in sigma2 for comparison to the online version

In [ ]:
%%time
### AMOCz in CM4hires
ComputeDiag_dict_OfflineSig2 = {}
dmgetout_flag = False
zarr_dir='/xtmp/Matthew.Thomas/work'
if dmgetout_flag: print(f'run the following dmget commands:')
for keys in run_dict:
    ds, grid, moc_sig, rhopoto = MOC_funcs.MOC_basin_latrange_offline(dir_vars=run_dict[keys]['depth_vars_path'],dir_grid=run_dict[keys]['grid_path'],file_str_identifier=run_dict[keys]['file_str_identifier'],lat_range=run_dict[keys]['lat_range'],basin_list=run_dict[keys]['basin_list'],rholims=[20,39],ref_pres=2000,z_layer_var=run_dict[keys]['z_layer_var'],z_inter_var=run_dict[keys]['z_inter_var'],decode_times_flag=True,dmgetout=dmgetout_flag,zarr_dir=zarr_dir)
    if len(ds)>0:
        moc_z=ds.vmo.mean('time').fillna(0).sum('xh').cumsum('z_l')/1e6/1030    
        ComputeDiag_dict_OfflineSig2[keys]={
                                'ds' : ds, 'grid': grid, 'moc_sig' : moc_sig, 'rhopoto' : rhopoto, 'moc_z' : moc_z,
                                }

In [ ]:
%%time
load_data_flag=1
fig = plt.figure(figsize=(16,len(ComputeDiag_dict_OfflineSig2.keys())*5))
subplot_num=1
for keys in run_dict:
    
    if load_data_flag:
        ComputeDiag_dict_OfflineSig2[keys]['moc_sig']=ComputeDiag_dict_OfflineSig2[keys]['moc_sig'].load()
        ComputeDiag_dict_OfflineSig2[keys]['moc_z']=ComputeDiag_dict_OfflineSig2[keys]['moc_z'].load()
    
    ax = fig.add_subplot(len(ComputeDiag_dict_OfflineSig2.keys()),2,(subplot_num*2)-1)
    ComputeDiag_dict_OfflineSig2[keys]['moc_sig'].plot.contourf(y='rhopoto_bin',vmin=-10,vmax=24,levels=30,ylim=[38,30],cmap='RdBu_r')
    plt.title('AMOCsig for years '+run_dict[keys]['file_str_identifier']+'. Run '+keys)
    plt.xlabel('')
    ax = fig.add_subplot(len(ComputeDiag_dict_OfflineSig2.keys()),2,subplot_num*2)
    ComputeDiag_dict_OfflineSig2[keys]['moc_z'].plot.contourf(vmin=-5,vmax=20,levels=30,ylim=[6000,0],xlim=[run_dict[keys]['lat_range'][0],run_dict[keys]['lat_range'][1]],cmap='RdBu_r')
    plt.fill_between(ComputeDiag_dict_OfflineSig2[keys]['grid'].yh,ComputeDiag_dict_OfflineSig2[keys]['grid']['deptho'].max(dim='xh'),8000,color='k')
    plt.title('AMOCz for years '+run_dict[keys]['file_str_identifier']+'. Run '+keys)
    plt.xlabel('')
    subplot_num+=1

In [ ]:
cluster.close()
client.close()

In [ ]:
# done! 